In [13]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [14]:
import pandas as pd
import mappings as mappings
import schemas as schemas
import datetime

current_year = 2023
accounts_return_period_start_date = datetime.date(current_year - 1, 9, 10)
academy_year_start_date = datetime.date(current_year - 1, 9, 1)
academy_year_end_date = datetime.date(current_year, 8, 30)
maintained_schools_year_start_date = datetime.date(current_year, 4, 1)
maintained_schools_year_end_date = datetime.date(current_year, 3, 31)

## GIAS data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [15]:
gias = pd.read_csv('data/edubasealldata20240312.csv', encoding='cp1252', 
                   index_col=schemas.gias_index_col, usecols=schemas.gias.keys(), dtype=schemas.gias)

gias_links = pd.read_csv('data/links_edubasealldata20240312.csv', encoding='cp1252', 
                         index_col=schemas.gias_links_index_col, usecols=schemas.gias_links.keys(), dtype=schemas.gias_links)

# GIAS transformations
gias['LA Establishment Number'] = gias['LA (code)'] + '-' + gias['EstablishmentNumber'].astype('string')
gias['LA Establishment Number'] = gias['LA Establishment Number'].astype('string')

gias['OpenDate'] = pd.to_datetime(gias['OpenDate'], dayfirst=True, format='mixed')
gias['CloseDate'] = pd.to_datetime(gias['CloseDate'], dayfirst=True, format='mixed')
gias['SchoolWebsite'] = gias['SchoolWebsite'].fillna('').map(mappings.map_school_website)
gias['Boarders (name)'] = gias['Boarders (name)'].fillna('').map(mappings.map_boarders)
gias['OfstedRating (name)'] = gias['OfstedRating (name)'].fillna('').map(mappings.map_ofsted_rating)
gias['NurseryProvision (name)'] = gias['NurseryProvision (name)'].fillna('')
gias['OfficialSixthForm (name)'] = gias['OfficialSixthForm (name)'].fillna('').map(mappings.map_sixth_form)
gias['AdmissionsPolicy (name)'] = gias['AdmissionsPolicy (name)'].fillna('').map(mappings.map_admission_policy)
gias['Head Name'] = gias['HeadTitle (name)'] + ' ' + gias['HeadFirstName'] + ' ' + gias['HeadLastName']

In the following cell, we find all the predecessor and merged links. The links are then Ranked by URN and order by 'Link Established Date'. The linked GAIS data in then joined to the base GIAS data. This creates the overall school data set. This dataset is then filtered for schools that are open (CloseDate is null) and the schools with nested links that are Ranked 1.

In [16]:
gias_links = gias_links[
    gias_links['LinkType'].isin(['Predecessor', 
                                 'Predecessor - amalgamated', 
                                 'Predecessor - Split School', 
                                 'Predecessor - merged', 
                                 'Merged - expansion of school capacity', 
                                 'Merged - change in age range'])
].sort_values(by='LinkEstablishedDate', ascending=False)

gias_links['Rank'] = gias_links.groupby('URN').cumcount() + 1
gias_links['Rank'] = gias_links['Rank'].astype('Int64')

schools = gias.join(gias_links, on='URN', how='left', rsuffix='_links', lsuffix='_school').sort_values(by='URN')

schools = schools[
    schools['CloseDate'].isna() & ((schools['Rank'] == 1) | (schools['Rank'].isna()))
]

In [17]:
schools.sort_index()

,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentStatus (code),EstablishmentStatus (name),OpenDate,CloseDate,...,OfstedRating (name),MSOA (code),LSOA (code),LA Establishment Number,Head Name,LinkURN,LinkName,LinkType,LinkEstablishedDate,Rank
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,3614,The Aldgate School,2,Voluntary aided school,1,Open,NaT,NaT,...,Outstanding,E02000001,E01032739,201-3614,Miss Alexandra Allan,<NA>,<NA>,<NA>,<NA>,<NA>
100001,201,City of London,6005,City of London School for Girls,11,Other independent school,1,Open,1920-01-01,NaT,...,,E02000001,E01000002,201-6005,Mrs Jenny Brown,<NA>,<NA>,<NA>,<NA>,<NA>
100002,201,City of London,6006,St Paul's Cathedral School,11,Other independent school,1,Open,1939-01-01,NaT,...,,E02000001,E01032739,201-6006,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
100003,201,City of London,6007,City of London School,11,Other independent school,1,Open,1919-01-01,NaT,...,,E02000001,E01032739,201-6007,Mr Alan Bird,<NA>,<NA>,<NA>,<NA>,<NA>
100005,202,Camden,1048,Thomas Coram Centre,15,Local authority nursery school,1,Open,NaT,NaT,...,Outstanding,E02007115,E01000937,202-1048,Ms Perina Holness,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402468,679,Monmouthshire,5500,King Henry viii 3-19 School,30,Welsh establishment,1,Open,2023-09-01,NaT,...,,999999999,999999999,679-5500,<NA>,402093,Deri View Primary,Predecessor - amalgamated,31-08-2023,1
402469,681,Cardiff,2333,Ysgol Gynradd Groes-Wen Primary,30,Welsh establishment,1,Open,2023-09-01,NaT,...,,W02000380,W01001729,681-2333,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
402470,668,Pembrokeshire,2398,Ysgol Bro Penfro,30,Welsh establishment,4,Proposed to open,2024-09-01,NaT,...,,W02000140,W01000607,668-2398,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## CDC data load and preparation

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [18]:
cdc = pd.read_csv('data/standing_data_cdc.csv', encoding='utf8', index_col=schemas.cdc_index_col, usecols=schemas.cdc.keys(), dtype=schemas.cdc)

cdc['Total Area'] = cdc.groupby(by=['URN'])['GIFA'].sum()
cdc['Proportion Area'] = (cdc['GIFA'] / cdc['Total Area'])
cdc['Indicative Age'] = cdc['Block Age'].fillna('').map(mappings.map_block_age).astype('Int64')
cdc['Age Score'] = cdc['Proportion Area'] * (current_year - cdc['Indicative Age'])
cdc['Score'] = cdc.groupby(by=['URN'])['Age Score'].sum()
cdc = cdc[['Total Area', 'Score']].drop_duplicates().sort_index()

In [19]:
cdc

,Total Area,Score
URN,,
100000,3628.0,143.0
100006,1523.0,60.797111
100008,2951.0,118.0
100009,3021.0,135.619662
100010,3415.0,143.0
...,...,...
149314,2626.0,133.451257
149363,1107.0,70.407407
149431,2120.0,143.0


## School Census data load

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets. There is a lot of duplicated columns in the datasets, these are then removed with preference for the pupil census columns.

In [20]:
school_workforce_census = pd.read_csv('data/School_Tables_School_Workforce_Census_2022.csv', encoding='utf8', index_col=schemas.workforce_census_index_col, usecols=schemas.workforce_census.keys(), dtype=schemas.workforce_census, na_values=["x","u"], keep_default_na=True).drop_duplicates()

school_pupil_census = pd.read_csv('data/standing_data_census_pupils.csv', encoding='utf8', index_col=schemas.pupil_census_index_col, usecols=schemas.pupil_census.keys(), dtype=schemas.pupil_census).drop_duplicates()

school_census = school_pupil_census.join(school_workforce_census, on='URN', how='inner', rsuffix='_pupil', lsuffix='_workforce')
                 

In [21]:
school_census

,headcount of pupils,% of pupils known to be eligible for and claiming free school me,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Full-Time Equivalent),Total Number of Teachers (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Number of Vacant Teacher Posts
URN,,,,,,,
141334,325.0,33.8,93.0,34.2,13.1,24.8,0.0
141396,642.0,23.4,236.0,82.5,34.0,18.3,0.0
141397,500.0,33.2,127.0,72.8,24.6,19.7,0.0
142223,1117.0,5.1,343.0,99.7,47.1,23.0,0.0
144396,217.0,56.7,29.0,25.6,11.4,18.1,0.0
...,...,...,...,...,...,...,...
104642,421.0,2.4,14.0,34.5,15.8,26.6,0.0
104643,432.0,3.5,13.0,39.9,17.4,24.7,0.0
104645,237.0,32.9,43.0,26.5,12.4,19.1,0.0


## Special Education Needs (SEN) data load and preparation

This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [22]:
sen = pd.read_csv('data/SEN.csv', encoding='cp1252', index_col=schemas.sen_index_col, dtype=schemas.sen, usecols=schemas.sen.keys())

In [23]:
sen

,Total pupils,SEN support,EHC plan,EHC_Primary_need_spld,EHC_Primary_need_mld,EHC_Primary_need_sld,EHC_Primary_need_pmld,EHC_Primary_need_semh,EHC_Primary_need_slcn,EHC_Primary_need_hi,...,SUP_Primary_need_pmld,SUP_Primary_need_semh,SUP_Primary_need_slcn,SUP_Primary_need_hi,SUP_Primary_need_vi,SUP_Primary_need_msi,SUP_Primary_need_pd,SUP_Primary_need_asd,SUP_Primary_need_oth,SUP_Primary_need_nsa
URN,,,,,,,,,,,,,,,,,,,,,
100000,271,59,8,0,0,0,0,1,1,0,...,0,8,30,2,0,2,0,4,0,7
100001,739,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100002,269,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100003,1045,145,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100005,136,23,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,21,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,41,2,3,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
149632,1291,136,58,2,1,0,0,3,15,4,...,0,17,10,4,2,1,3,6,16,34
149633,86,7,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0


## Academies data load and preparation 

This read academy data from the `master_list_raw` data file and joins on the GIAS Data (schools) school_census and CDC data by the 'LA Establishment Number', 'URN' and 'URN' keys respectively. Finally, some further mapping of the Nursery provision and the Academy status is carried out. 

In [24]:
academies_list = pd.read_csv('data/master_list_raw.csv', encoding='utf8', index_col=schemas.academy_master_list_index_col, dtype=schemas.academy_master_list, usecols=schemas.academy_master_list.keys())

academies = academies_list.merge(schools.reset_index(), left_index=True, right_on='LA Establishment Number')
academies.set_index('URN', inplace=True)

academies = (academies.join(school_census, how='left', rsuffix='_census', lsuffix='_academy')
             .join(cdc, on='URN', how='left', rsuffix='_cdc', lsuffix='_academy'))

academies['Type of Provision - Phase'] = academies.apply(lambda df: mappings.map_academy_phase_type(df['TypeOfEstablishment (code)'], df['Type of Provision - Phase']), axis=1)

# Bizarre I shouldn't need this as this is coming from the original GIAS dataset, but I seem to have to do this twice. 
academies['NurseryProvision (name)'] = academies['NurseryProvision (name)'].fillna('')
academies['NurseryProvision (name)'] = academies.apply(lambda df: mappings.map_nursery(df['NurseryProvision (name)'], df['Type of Provision - Phase']), axis=1)

academies['Status'] = academies.apply(lambda df: mappings.map_academy_status(pd.to_datetime(df['Date left or closed if in period']), 
                                                                             pd.to_datetime(df['Valid to']), 
                                                                             pd.to_datetime(df['OpenDate']), 
                                                                             pd.to_datetime(df['CloseDate']), 
                                                                             pd.to_datetime(accounts_return_period_start_date), pd.to_datetime(academy_year_start_date), pd.to_datetime(academy_year_end_date)), axis=1)


In [25]:
academies_list.sort_index()

,Company Registration Number,Incorporation Date,Academy Trust UPIN,UKPRN,Academy Trust Name,Academy Name,Academy UPIN,Trust Type,Date Opened,LA Name,...,Regional School Commissioner,Valid From,Valid to,Date joined or opened if in period,Date left or closed if in period,Territory,Academy Status,Academy Trust Status,Number of Academies in Trust,Number of Pupils
LA Establishment Number,,,,,,,,,,,,,,,,,,,,,
-,09999999,2011-02-28 00:00:00.0000000,128400,90000118,Coketown Academy Trust,North Coketown Primary School,999998,Multi Academy Trust (MAT),2014-09-01 00:00:00.0000000,Coketown County Council,...,Central and East,2021-09-01 00:00:00.0000000,<NA>,<NA>,<NA>,Central and East,Member for whole period,Existing,8,301
202-2000,07451568,2010-11-25 00:00:00.0000000,135636,10058418,St Luke's Church of England School,St Luke's Church of England School - Camden,120536,Single Academy Trust (SAT),2011-09-01 00:00:00.0000000,Camden London Borough Council,...,North West London & South Central,2011-09-01 00:00:00.0000000,<NA>,<NA>,<NA>,South Central and East,Member for whole period,Existing,1,102
202-2001,07468210,2010-12-13 00:00:00.0000000,135716,10058447,Anthem Schools Trust,Abacus Belsize Primary School,123288,Multi Academy Trust (MAT),2013-09-01 00:00:00.0000000,Camden London Borough Council,...,North West London & South Central,2013-09-01 00:00:00.0000000,<NA>,<NA>,<NA>,South Central and East,Member for whole period,Existing,16,163
202-2002,08803983,2013-12-05 00:00:00.0000000,137749,10060691,King's Cross Academy Trust,Kings Cross Academy,130943,Single Academy Trust (SAT),2015-09-01 00:00:00.0000000,Camden London Borough Council,...,North West London & South Central,2015-09-01 00:00:00.0000000,<NA>,<NA>,<NA>,South Central and East,Member for whole period,Existing,1,410
202-4000,07024902,2009-09-21 00:00:00.0000000,138179,10058277,The UCL Academy,The UCL Academy,120181,Single Academy Trust (SAT),2012-09-01 00:00:00.0000000,Camden London Borough Council,...,North West London & South Central,2012-09-01 00:00:00.0000000,<NA>,<NA>,<NA>,South Central and East,Member for whole period,Existing,1,1172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999-9993,09999999,2011-02-28 00:00:00.0000000,128400,90000118,Coketown Academy Trust,Coketown High School,999993,Multi Academy Trust (MAT),2016-11-01 00:00:00.0000000,Coketown County Council,...,Central and East,2021-11-01 00:00:00.0000000,<NA>,2021-11-01 00:00:00.0000000,<NA>,Central and East,in period transfer,Existing,8,225
999-9994,09999999,2011-02-28 00:00:00.0000000,128400,90000118,Coketown Academy Trust,Coketown Road Primary School,999994,Multi Academy Trust (MAT),2014-06-01 00:00:00.0000000,Coketown County Council,...,Central and East,2022-06-01 00:00:00.0000000,<NA>,<NA>,<NA>,Central and East,Member for whole period,Existing,8,205
999-9995,09999999,2011-02-28 00:00:00.0000000,128400,90000118,Coketown Academy Trust,Coketown West High School,999995,Multi Academy Trust (MAT),2011-04-01 00:00:00.0000000,Coketown County Council,...,Central and East,2022-04-01 00:00:00.0000000,<NA>,<NA>,<NA>,Central and East,Member for whole period,Existing,8,312


## Maintained schools data load and preparation

In [26]:
maintained_schools_list = pd.read_csv('data/maintained_schools_raw.csv', encoding='utf8', index_col=schemas.maintained_schools_master_list_index_col, usecols=schemas.maintained_schools_master_list.keys(), dtype=schemas.maintained_schools_master_list)

maintained_schools = maintained_schools_list.merge(schools.reset_index(), left_index=True, right_on='URN')
maintained_schools = (maintained_schools.join(school_census, how='left', rsuffix='_census', lsuffix='_school')
                      .join(cdc, on='URN', how='left', rsuffix='_cdc', lsuffix='_academy'))

maintained_schools['Status'] = maintained_schools.apply(lambda df: mappings.map_maintained_school_status(df['OpenDate'], df['CloseDate'], df['Period covered by return (months)'], pd.to_datetime(maintained_schools_year_start_date), pd.to_datetime(maintained_schools_year_end_date)), axis=1)

maintained_schools.set_index('URN', inplace=True)

In [27]:
maintained_schools

,LA,LA Name,Region,Estab,LAEstab,School Name,Phase,Overall Phase,Lowest age of pupils,Highest age of pupils,...,headcount of pupils,% of pupils known to be eligible for and claiming free school me,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Full-Time Equivalent),Total Number of Teachers (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Number of Vacant Teacher Posts,Total Area,Score,Status
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,London,3614,2013614,The Aldgate School,Infant and junior,Primary,3.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3628.0,143.0,Open
100005,202,Camden,London,1048,2021048,Thomas Coram Centre,Nursery,Nursery,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Open
100006,202,Camden,London,1100,2021100,Heath School,Pupil referral unit,Pupil referral unit,11.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1523.0,60.797111,Open
100007,202,Camden,London,1101,2021101,Camden Primary Pupil Referral Unit,Pupil referral unit,Pupil referral unit,5.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,Open
100008,202,Camden,London,2019,2022019,Argyle Primary School,Infant and junior,Primary,3.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2951.0,118.0,Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131818,341,Liverpool,North West,2230,3412230,Fazakerley Primary School,Infant and junior,Primary,3.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2819.0,57.148634,Open
132176,341,Liverpool,North West,2232,3412232,Kirkdale St Lawrence CofE VA Primary School,Infant and junior,Primary,3.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2872.0,28.0,Open
132793,341,Liverpool,North West,2233,3412233,St Matthew's Catholic Primary School,Infant and junior,Primary,4.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1639.0,118.0,Open
